In [1]:
### 형태소 분석(knolPy) 라이브러리 설치
! pip install konlpy

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 19.4/19.4 MB 8.6 MB/s eta 0:00:00
     -------------------------------------- 344.5/344.5 kB 7.1 MB/s eta 0:00:00


In [2]:
### 사용
from konlpy.tag import Twitter
twitter = Twitter()
malist = twitter.pos('아버지가 방에 들어가신다',norm=True,stem=True)
malist

[('아버지', 'Noun'),
 ('가', 'Josa'),
 ('방', 'Noun'),
 ('에', 'Josa'),
 ('들어가다', 'Verb')]

In [3]:
### 사용
from konlpy.tag import Twitter,Okt
twitter = Okt()
malist = twitter.pos('아버지가 방에 들어가신다',norm=True,stem=True)
malist

[('아버지', 'Noun'),
 ('가', 'Josa'),
 ('방', 'Noun'),
 ('에', 'Josa'),
 ('들어가다', 'Verb')]

In [7]:
### 사용
from konlpy.tag import Twitter,Okt
twitter = Okt()
malist = twitter.pos('그래욬ㅋㅋ?',norm=True,stem=True)
malist

[('그렇다', 'Adjective'), ('ㅋㅋ', 'KoreanParticle'), ('?', 'Punctuation')]

In [22]:
### Twitter를 이용한 카운팅
from konlpy.tag import Twitter

fp = open('data_text/토지1.txt','r',encoding='utf-8')
text = fp.read()
# print(text)

## 텍스트 한줄 처리
twitter = Twitter()
word_dic = {}
lines = text.split('\n')
for line in lines:
    malist = twitter.pos(line)
    for word in malist:
        if word[1] == 'Noun':   # 명사 찾기 조건
            if not (word[0] in word_dic):
                word_dic[word[0]] = 0
            word_dic[word[0]] += 1      # 카운트 처리

## 정렬 : 가장 많이 사용된 명사부터 출력
keys = sorted(word_dic.items(),key=lambda x:x[1], reverse=True)

## 많이 사용된 명사 출력
for word, count in keys[:50]:
    print('{0}({1}) '.format(word,count),end="")
print()

C:\Users\user\AppData\Roaming\Python\Python310\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


것(1030) 그(896) 말(833) 안(445) 용이(344) 눈(323) 평산(320) 사람(315) 내(303) 놈(295) 이(289) 길(278) 소리(272) 일(268) 거(257) 얼굴(255) 생각(242) 못(241) 수(234) 치수(228) 집(222) 나(222) 댁(221) 때(213) 네(206) 강청댁(198) 머(193) 봉(187) 서방(185) 년(183) 최(174) 제(173) 더(172) 어디(164) 강(159) 마을(149) 니(148) 날(140) 뒤(138) 기(136) 칠성(136) 포수(136) 녀(135) 김(135) 임(134) 양반(132) 아이(131) 서희(131) 하나(123) 속(122) 


In [ ]:
### Word2Vec으로 문장을 벡터로 변환

In [23]:
# 라이브러리 있는지 확인
!pip list | findstr gensim

gensim               4.2.0


In [26]:
## Gensim의 Word2Vec로 '토지'읽어오기
from konlpy.tag import Twitter
from gensim.models import word2vec

# 토지 읽어오기
fp = open('data_text/토지1.txt','r',encoding='utf-8')
text = fp.read()

# 텍스트를 한줄씩 처리하기
twitter = Twitter()
results = []
lines = text.split('\n')
for line in lines:
    # 형태소 분석하기
    # 단어의 기본형 사용
    malist = twitter.pos(line,norm=True,stem=True)
    r = []
    for word in malist:
        # 어미/조사/구두점 등은 대상에서 제외
        if not word[1] in ['Josa','Eomi','Punctuation']:
            r.append(word[0])
    rl = (" ".join(r)).strip()
    results.append(rl)
    # print(rl)

# 파일로 출력하기
wakati_file = 'data_text/toji.wakati'
with open(wakati_file,'w',encoding='utf-8') as fp:
    fp.write('\n'.join(results))

# Word2Vec 모델 만들기
data = word2vec.LineSentence(wakati_file)
model = word2vec.Word2Vec(data,hs=1,min_count=2,sg=1,vector_size=200,window=10)
model.save("data_text/toji.model")
print('ok')

C:\Users\user\AppData\Roaming\Python\Python310\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


ok


In [49]:
model = word2vec.Word2Vec.load('data_text/toji.model')
model.wv.most_similar(positive=['종'])

[('하인', 0.8215736746788025),
 ('최씨', 0.8038226962089539),
 ('구걸', 0.8036032915115356),
 ('천하다', 0.7992435693740845),
 ('서성대다', 0.7948043346405029),
 ('절간', 0.7935255765914917),
 ('계집', 0.791433572769165),
 ('어무님', 0.7888969779014587),
 ('들고나다', 0.7888922691345215),
 ('피장파장', 0.78848797082901)]

In [ ]:
### wiki.txt
from konlpy.tag import Twitter, Okt
from gensim.models import word2vec

# 파일열기
read_file = open('data_text/wiki.txt','r',encoding='utf-8')
wakati_file = "data_text/wiki.wakati"
write_file = open(wakati_file,'w',encoding='utf-8')

# 형태소 분석
twitter = Twitter()
i=0
# 텍스트를 한 줄씩 처리하기
while True:
    line = read_file.readline()
    if not line: break
    if i % 20000 == 0:
        print('current - ' + str(i))
    i += 1
    # 형태소 분석
    malist = twitter.pos(line, norm=True, stem=True)
    # 필요한 어구만 대상으로 하기
    r = []
    for word in malist:
        # 어미/조사/구두점 등은 대상에서 제외
        if not word[1] in ['Josa','Eomi','Punctuation']:
            write_file.write(word[0] + " ")
write_file.close()

In [1]:
from gensim.models import word2vec
data = word2vec.Text8Corpus('data_text/wiki.wakati')
model = word2vec.Word2Vec(data, vector_size=100)
model.save('data_text/wiki.model')
print('ok')

ok
